In [55]:
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import duckdb

In [56]:
equipos = pd.read_csv("./enunciado_tablas/enunciado_equipos.csv")
jugadores_atributos = pd.read_csv("./enunciado_tablas/enunciado_jugadores_atributos.csv")
jugadores = pd.read_csv("./enunciado_tablas/enunciado_jugadores.csv")
liga = pd.read_csv("./enunciado_tablas/enunciado_liga.csv")
paises = pd.read_csv("./enunciado_tablas/enunciado_paises.csv")
partidos = pd.read_csv("./enunciado_tablas/enunciado_partidos.csv")

In [57]:
# limpio y renombro las columnas del dataframe 'jugadores'
jugadores_ok = jugadores.dropna()
jugadores_ok.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11060 entries, 0 to 11059
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player_api_id  11060 non-null  int64  
 1   player_name    11060 non-null  object 
 2   birthday       11060 non-null  object 
 3   height         11060 non-null  float64
 4   weight         11060 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 432.2+ KB


Creo la tabla "Jugador"

In [58]:
Jugador = jugadores_ok[["player_api_id", "player_name", "height", "weight"]]
Jugador = Jugador.rename(columns={"player_api_id":"Id_Jugador", "player_name":"Nombre", "height":"Altura", "weight":"Peso"})
Jugador

,Id_Jugador,Nombre,Altura,Peso
0,505942,Aaron Appindangoye,182.88,187
1,155782,Aaron Cresswell,170.18,146
2,162549,Aaron Doran,170.18,163
3,30572,Aaron Galindo,182.88,198
4,23780,Aaron Hughes,182.88,154
...,...,...,...,...
11055,26357,Zoumana Camara,182.88,168
11056,111182,Zsolt Laczko,182.88,176
11057,36491,Zsolt Low,180.34,154
11058,35506,Zurab Khizanishvili,185.42,172


In [59]:
# limpio los atributos necesarios
jugadores_atributos_ok = jugadores_atributos.dropna()
jugadores_atributos_ok["date"] = pd.to_datetime(jugadores_atributos_ok["date"])
Jugador_Atributos_Variables = jugadores_atributos_ok[["player_api_id","potential","sprint_speed", "date"]]
Jugador_Atributos_Variables = Jugador_Atributos_Variables.rename(columns={
    "player_api_id": "Id_Jugador",
    "potential": "Potencial",
    "sprint_speed": "Velocidad",
    "date": "Fecha"
})

Jugador_Atributos_Variables

C:\Users\nahue\AppData\Local\Temp\ipykernel_4176\494634233.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jugadores_atributos_ok["date"] = pd.to_datetime(jugadores_atributos_ok["date"])


,Id_Jugador,Potencial,Velocidad,Fecha
0,505942,71.0,64.0,2016-02-18
1,505942,71.0,64.0,2015-11-19
2,505942,66.0,64.0,2015-09-21
3,505942,65.0,64.0,2015-03-20
4,505942,65.0,64.0,2007-02-22
...,...,...,...,...
183973,39902,85.0,72.0,2009-08-30
183974,39902,80.0,67.0,2009-02-22
183975,39902,80.0,67.0,2008-08-30
183976,39902,81.0,60.0,2007-08-30


In [60]:
# junto las tablas con un merge inner (similar al inner join de SQL)

Jugador = pd.merge(Jugador, Jugador_Atributos_Variables, how="inner", on="Id_Jugador")
Jugador

,Id_Jugador,Nombre,Altura,Peso,Potencial,Velocidad,Fecha
0,505942,Aaron Appindangoye,182.88,187,71.0,64.0,2016-02-18
1,505942,Aaron Appindangoye,182.88,187,71.0,64.0,2015-11-19
2,505942,Aaron Appindangoye,182.88,187,66.0,64.0,2015-09-21
3,505942,Aaron Appindangoye,182.88,187,65.0,64.0,2015-03-20
4,505942,Aaron Appindangoye,182.88,187,65.0,64.0,2007-02-22
...,...,...,...,...,...,...,...
181260,39902,Zvjezdan Misimovic,180.34,176,85.0,72.0,2009-08-30
181261,39902,Zvjezdan Misimovic,180.34,176,80.0,67.0,2009-02-22
181262,39902,Zvjezdan Misimovic,180.34,176,80.0,67.0,2008-08-30
181263,39902,Zvjezdan Misimovic,180.34,176,81.0,60.0,2007-08-30


In [66]:
consulta = """
    SELECT Potencial, Velocidad, Fecha 
    FROM Jugador_Atributos_Variables
    WHERE Id_Jugador = 39902
"""
res = duckdb.sql(consulta).df()
res

,Potencial,Velocidad,Fecha
0,78.0,63.0,2012-08-31
1,81.0,63.0,2012-02-22
2,81.0,63.0,2011-08-30
3,85.0,69.0,2011-02-22
4,85.0,69.0,2010-08-30
5,85.0,72.0,2009-08-30
6,80.0,67.0,2009-02-22
7,80.0,67.0,2008-08-30
8,81.0,60.0,2007-08-30
9,81.0,64.0,2007-02-22
